# Cummulative Industry Exposure

In [1]:
# Import libraries

# System

import os

# Analysis

import numpy as np # Numerical Analysis
import pandas as pd # Data Mgmt
import geopandas as gpd # Spatial Data Mgmt
import rasterio # Rasters
from rasterio import features

In [2]:
# Load Data

# Minneapolis Boundary

mpls_path = os.path.join(os.getcwd(), '..', '1_Data_IO', 'Data', 'mpls_boundary.geojson')
mpls = gpd.read_file(mpls_path)

# Average Annual Daily Traffic (AADT)

aadt_path = os.path.join(os.getcwd(), '..', '1_Data_IO', 'Data', 'aadt.geojson')
aadt = gpd.read_file(aadt_path)

# Permitted Emissions

emissions_path = os.path.join(os.getcwd(), '..', '1_Data_IO', 'Data', 'emissions.csv')
emissions = pd.read_csv(emissions_path)
emissions_geo = gpd.GeoDataFrame(emissions,
                                 crs = 'EPSG:26915',
                                 geometry = gpd.points_from_xy(x=emissions.EASTING,
                                                                 y=emissions.NORTHING)
                                )

## Prep

### Clip

In [3]:
# Study is everything within 8 km of minneapolis

mpls_buffed = mpls.geometry.buffer(8000)

emissions_select = gpd.clip(emissions_geo, mpls_buffed)

aadt_select = gpd.clip(aadt, mpls_buffed)

/home/robb/miniconda3/envs/gis5571/lib/python3.10/site-packages/geopandas/tools/clip.py:66: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  clipped.loc[


### Define Layers

In [4]:
# 1) AADT is only current values for now

# 2) Emissions can be current and cummulative

# Pollutants of interest: 'Sulfur Dioxide', 'PM Primary', 'PM10 Primary', 'PM2.5 Primary',
# 'Volatile Organic Compounds'] 
# But there's also all the dioxins... "Plastics" that are measured every 5 years or so

pollutant_names = ['Sulfur Dioxide', 'PM Primary', 'PM10 Primary', 'PM2.5 Primary', 'Volatile Organic Compounds']

## Rasterize Using Rasterio

In [5]:
# Get metadata from template
rst = rasterio.open('template.tif') # Open template
meta = rst.meta.copy() # Copy template metadata
# meta.update(compress='lzw') # Good for integers/categorical rasters
rst.close()

### AADT

In [6]:
# Filenames
out_fn = os.path.join('Rasterized_Sources','rasterized_aadt.tif')

with rasterio.open(out_fn, 'w+', **meta) as out: # Burn features into raster
    out_arr = out.read(1)
    
    aadt_sorted = aadt_select.sort_values('CURRENT_VO') # Sort by current vo for burning highest value

    # this is where we create a generator of geom, value pairs to use in rasterizing
    shapes = ((geom,value) for geom, value in zip(aadt_sorted.geometry, aadt_sorted.CURRENT_VO))

    burned = features.rasterize(shapes=shapes, 
                                fill=0, 
                                out=out_arr, 
                                transform=out.transform)#,
                                # merge_alg=rasterio.enums.MergeAlg.add) # This didn't work, it doubled up on vertices
                                        # ^Cummulative AADT if roads on top of eachother
                                        # Instead we just use highest value for each pixel
    out.write_band(1, burned)

### Cummulative Emissions

In [7]:
for pollutant_name in pollutant_names:
    
    # Select data
    pollutant_select = emissions_select[emissions_select.POLLUTANT == pollutant_name]
    
    # Filenames
    out_fn = os.path.join('Rasterized_Sources','rasterized_' + pollutant_name + '_cummulative.tif')

    with rasterio.open(out_fn, 'w+', **meta) as out: # Burn features into template raster
            
        out_arr = out.read(1)

        # this is where we create a generator of geom, value pairs to use in rasterizing
        shapes = ((geom,value) for geom, value in zip(pollutant_select.geometry, pollutant_select.LBS_EMITTED))

        burned = features.rasterize(shapes=shapes, 
                                    fill=0, 
                                    out=out_arr, 
                                    transform=out.transform,
                                    merge_alg=rasterio.enums.MergeAlg.add) 
                                            # ^Cummulative emissions if facilities on top of eachother
        out.write_band(1, burned)

### Current

In [8]:
current_emissions_select = emissions_select[emissions_select.YEAR == 2020]

In [9]:
for pollutant_name in pollutant_names:
    
    # Select data
    current_pollutant_select = current_emissions_select[current_emissions_select.POLLUTANT == pollutant_name]
    
    # Filenames
    out_fn = os.path.join('Rasterized_Sources','rasterized_' + pollutant_name + '_current.tif')

    with rasterio.open(out_fn, 'w+', **meta) as out: # Burn features into template raster
            
        out_arr = out.read(1)

        # this is where we create a generator of geom, value pairs to use in rasterizing
        shapes = ((geom,value) for geom, value in zip(current_pollutant_select.geometry, current_pollutant_select.LBS_EMITTED))

        burned = features.rasterize(shapes=shapes, 
                                    fill=0, 
                                    out=out_arr, 
                                    transform=out.transform,
                                    merge_alg=rasterio.enums.MergeAlg.add) 
                                            # ^Cummulative emissions if facilities on top of eachother
        out.write_band(1, burned)